In [1]:
GEN_BACK_TR = True

GEN_UPSAMPLE = False

GEN_EN_ONLY = False

In [2]:
#python basics
from matplotlib import pyplot as plt
import math, os, re, time
import numpy as np, pandas as pd, seaborn as sns
from multiprocessing import Pool
#nlp augmentation
!pip install --quiet googletrans==3.1.0a0
from googletrans import Translator

#model evaluation
from sklearn.model_selection import train_test_split, StratifiedKFold

#for fast parallel processing
from dask import bag, diagnostics

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 32.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 50.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 27.7 MB/s eta 0:0

In [ ]:
combined_dataset=pd.read_csv("/content/train.csv")

In [ ]:
def back_translate(sequence,label):
    languages = ['en', 'te','kn', 'es']
    #instantiate translator
    translator = Translator()
    output_sequence = [sequence]
    label_sequence = [label]
    #store original language so we can convert back
    org_lang = 'mr'
    #randomly choose language to translate sequence to  
    for language  in languages:
      #translate to new language and back to original
      translated = translator.translate(sequence, dest = language).text
      #translate back to original language
      translated_back = translator.translate(translated, dest = org_lang).text
      if translated_back not in output_sequence:
          output_sequence.append(translated_back)
          label_sequence.append(label)
    
    
    news_list.extend(output_sequence)
    label_list.extend(label_sequence)


#check performance
# output = back_translate('ನಾನು ಜಾಲಿ ಜೀವನ ನಡೆಸುತ್ತಿದ್ದೇನೆ')
# print(output)

In [ ]:
def back_translate_parallel(dataset):
    news = (dataset['headline']).tolist()
    labels = (dataset['label']).tolist()
    with diagnostics.ProgressBar():
        for i in range(0,len(news)):
          print(i)
          back_translate(news[i],labels[i])

In [ ]:
news_list = list()
label_list=list()
if GEN_BACK_TR:
#now we apply translation augmentation
    back_translate_parallel(combined_dataset)
    dic = {"news":news_list,"label":label_list}
    df = pd.DataFrame(dic)
    df.to_csv("Augmented_Train.csv",index=False)